In [5]:
# Part 1 Data Download

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("AmesHousing.csv", index_col=0)

# --- CHANGE: We DO NOT calculate the median or create the DV here. ---
# We keep SalePrice so we can calculate the median properly after splitting later.

# Display first few rows
df.head()

# Let's drop columns that are unnamed or have no meaning, and also drop PID
unnamed_cols = [col for col in df.columns if 'unnamed' in col.lower() or 'no meaning' in col.lower()]
columns_to_drop = unnamed_cols + ['PID']

# Drop identified columns
df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

print(f"Dropped {len(unnamed_cols)} unnamed columns and PID")
print(f"DataFrame shape after dropping columns: {df_cleaned.shape}")

# df_cleaned.head # corrected syntax below
print(df_cleaned.head())

Dropped 0 unnamed columns and PID
DataFrame shape after dropping columns: (2930, 80)
       MS SubClass MS Zoning  Lot Frontage  Lot Area Street Alley Lot Shape  \
Order                                                                         
1               20        RL         141.0     31770   Pave   NaN       IR1   
2               20        RH          80.0     11622   Pave   NaN       Reg   
3               20        RL          81.0     14267   Pave   NaN       IR1   
4               20        RL          93.0     11160   Pave   NaN       Reg   
5               60        RL          74.0     13830   Pave   NaN       IR1   

      Land Contour Utilities Lot Config  ... Pool Area Pool QC  Fence  \
Order                                    ...                            
1              Lvl    AllPub     Corner  ...         0     NaN    NaN   
2              Lvl    AllPub     Inside  ...         0     NaN  MnPrv   
3              Lvl    AllPub     Corner  ...         0     NaN    NaN

In [6]:
# Part 2 Missing Value Imputation

# Show the number of missing values before we start
print("Missing values before imputation:")
print(df_cleaned.isnull().sum().sort_values(ascending=False).head(10))
print("-" * 30)


df_sample1 = df_cleaned.copy()

# --- 1. Impute "Meaningful NA" Categoricals ---
meaningful_na_columns = [
    'Alley', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 
    'BsmtFin Type 1', 'BsmtFin Type 2', 'FireplaceQu',
    'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond',
    'Pool QC', 'Fence', 'Misc Feature', 'Mas Vnr Type'
]

for col in meaningful_na_columns:
    if col in df_sample1.columns:
        df_sample1[col] = df_sample1[col].fillna('None')

# --- 2. Numerical Imputation ---
df_sample2 = df_sample1.copy()

# A. Imputation (Context-Aware)

# Basement-related
if 'Bsmt Qual' in df_sample2.columns:
    mask = (df_sample2['Bsmt Qual'] == 'None')
    bsmt_num_cols = ['Total Bsmt SF', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Bsmt Full Bath', 'Bsmt Half Bath']
    for col in bsmt_num_cols:
        if col in df_sample2.columns:
            df_sample2.loc[mask, col] = df_sample2.loc[mask, col].fillna(0)

# Garage-related
if 'Garage Type' in df_sample2.columns:
    mask = (df_sample2['Garage Type'] == 'None')
    garage_num_cols = ['Garage Cars', 'Garage Area', 'Garage Yr Blt']
    for col in garage_num_cols:
        if col in df_sample2.columns:
            df_sample2.loc[mask, col] = df_sample2.loc[mask, col].fillna(0)

# Masonry veneer
if 'Mas Vnr Type' in df_sample2.columns:
    mask = (df_sample2['Mas Vnr Type'] == 'None')
    if 'Mas Vnr Area' in df_sample2.columns:
         df_sample2.loc[mask, 'Mas Vnr Area'] = df_sample2.loc[mask, 'Mas Vnr Area'].fillna(0)
 
# Lot Frontage (Neighborhood median)
if 'Lot Frontage' in df_sample2.columns and 'Neighborhood' in df_sample2.columns:
    df_sample2['Lot Frontage'] = df_sample2.groupby('Neighborhood')['Lot Frontage'].transform(lambda x: x.fillna(x.median()))
    df_sample2['Lot Frontage'] = df_sample2['Lot Frontage'].fillna(df_sample2['Lot Frontage'].median())

# C. Generic Median Imputation (Fallback)
all_numerical_cols = df_sample2.select_dtypes(include=np.number).columns
df_sample2[all_numerical_cols] = df_sample2[all_numerical_cols].fillna(value=df_sample2[all_numerical_cols].median())

# --- 3. Categorical Imputation ---
df_sample4 = df_sample2.copy()

# B. Generic Mode Imputation (Fallback)
all_categorical_cols = df_sample4.select_dtypes(include=['object', 'category']).columns

for col in all_categorical_cols:
     df_sample4[col] = df_sample4[col].fillna(df_sample4[col].mode()[0])

# --- 4. Final Check ---
total_missing = df_sample4.isnull().sum().sum()
print("-" * 30)
print(f"Total missing values remaining in df_sample4: {total_missing}")

Missing values before imputation:
Pool QC          2917
Misc Feature     2824
Alley            2732
Fence            2358
Mas Vnr Type     1775
Fireplace Qu     1422
Lot Frontage      490
Garage Cond       159
Garage Yr Blt     159
Garage Finish     159
dtype: int64
------------------------------
------------------------------
Total missing values remaining in df_sample4: 0


In [7]:
# Part 3 Variable transformation

# Lists defining predictors
nvar_list_original = [
    'Lot Frontage', 'Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2',
    'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
    'Gr Liv Area', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch',
    '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val',
    'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add', 'Bsmt Full Bath',
    'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr',
    'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt', 'Garage Cars', 'Mo Sold', 'Yr Sold'
]

# Removed 'InBudget' from here as it doesn't exist yet
cvar_list_original = [
    'MS SubClass', 'MS Zoning', 'Street', 'Alley', 'Land Contour', 'Lot Config',
    'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style',
    'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
    'Foundation', 'Heating', 'Central Air', 'Garage Type', 'Misc Feature',
    'Sale Type', 'Sale Condition', 
    'Lot Shape', 'Utilities', 'Land Slope', 'Exter Qual', 'Exter Cond',
    'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2',
    'Heating QC', 'Electrical', 'Kitchen Qual', 'Functional', 'Fireplace Qu',
    'Garage Finish', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence'
]

# --- CHANGE: Added SalePrice to model_cols so it carries over ---
model_cols = cvar_list_original + nvar_list_original + ['SalePrice']

df_sample5 = df_sample4[model_cols].copy()

# Standardize numerical variables (SalePrice is NOT in this list, so it stays raw)
df_sample5[nvar_list_original] = (df_sample4[nvar_list_original] - df_sample4[nvar_list_original].mean())/df_sample4[nvar_list_original].std()

# Set datatypes
df_sample6 = df_sample5.copy()
df_sample6[cvar_list_original] = df_sample5[cvar_list_original].astype('category')
df_sample6[nvar_list_original] = df_sample5[nvar_list_original].astype('float64')

# Convert categorical variables into dummies
df_sample7 = df_sample6.copy()
# Note: get_dummies ignores numericals like SalePrice, so it remains safe
df_sample7 = pd.get_dummies(df_sample6, prefix_sep='_', dtype=int)

# Remove redundant dummies (baseline)
# In more advanced code you might drop_first=True, but we stick to your logic
df_sample8 = df_sample7.copy()

print("\nRemaining columns after dummy reduction:")
print(df_sample8.columns.values)


Remaining columns after dummy reduction:
['Lot Frontage' 'Lot Area' 'Mas Vnr Area' 'BsmtFin SF 1' 'BsmtFin SF 2'
 'Bsmt Unf SF' 'Total Bsmt SF' '1st Flr SF' '2nd Flr SF' 'Low Qual Fin SF'
 'Gr Liv Area' 'Garage Area' 'Wood Deck SF' 'Open Porch SF'
 'Enclosed Porch' '3Ssn Porch' 'Screen Porch' 'Pool Area' 'Misc Val'
 'Overall Qual' 'Overall Cond' 'Year Built' 'Year Remod/Add'
 'Bsmt Full Bath' 'Bsmt Half Bath' 'Full Bath' 'Half Bath' 'Bedroom AbvGr'
 'Kitchen AbvGr' 'TotRms AbvGrd' 'Fireplaces' 'Garage Yr Blt'
 'Garage Cars' 'Mo Sold' 'Yr Sold' 'SalePrice' 'MS SubClass_20'
 'MS SubClass_30' 'MS SubClass_40' 'MS SubClass_45' 'MS SubClass_50'
 'MS SubClass_60' 'MS SubClass_70' 'MS SubClass_75' 'MS SubClass_80'
 'MS SubClass_85' 'MS SubClass_90' 'MS SubClass_120' 'MS SubClass_150'
 'MS SubClass_160' 'MS SubClass_180' 'MS SubClass_190' 'MS Zoning_A (agr)'
 'MS Zoning_C (all)' 'MS Zoning_FV' 'MS Zoning_I (all)' 'MS Zoning_RH'
 'MS Zoning_RL' 'MS Zoning_RM' 'Street_Grvl' 'Street_Pave' 'Alley

In [8]:
# Part 4 KNN Classification for Predicting “Not In Budget”

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn import metrics

# --- 1. Data Partition ---

# Select columns. We currently have Predictors + SalePrice.
# We split FIRST, then calculate the median on the training set.
df4partition = df_sample8.copy()

# Split into training and test sets (80/20 split)
df_nontestData, df_testData = train_test_split(
    df4partition, test_size=0.2, random_state=1
)

# --- 2. Create Dependent Variable  ---

# Calculate median ONLY on training data
train_median = df_nontestData['SalePrice'].median()
print(f"Median Sale Price (Training Set): ${train_median:,.2f}")

# Create DV for Training Set
df_nontestData['InBudget_Not in Budget'] = (df_nontestData['SalePrice'] > train_median).astype(int)

# Create DV for Test Set (using training median)
df_testData['InBudget_Not in Budget'] = (df_testData['SalePrice'] > train_median).astype(int)

# Define Target Variable Name
DV = "InBudget_Not in Budget"

# Define X and y 
# CRITICAL: Drop DV *and* SalePrice from predictors
X_train = df_nontestData.drop(columns=[DV, 'SalePrice'])
y_train = df_nontestData[DV]

X_test = df_testData.drop(columns=[DV, 'SalePrice'])
y_test = df_testData[DV]

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# --- 3. Fit Initial KNN Model (k=5) ---
k = 5
clf = KNeighborsClassifier(metric='euclidean', n_neighbors=k).fit(X_train, y_train)

# Evaluate AUC on test partition
y_pred_proba = clf.predict_proba(X_test)[:, 1]
auc_initial = roc_auc_score(y_test, y_pred_proba)
print(f"\nInitial KNN model (k={k}) AUC: {auc_initial:.3f}")

# --- 4. Cross-Validation for Optimal k ---

kfolds = 5
# Reducing max_k slightly to speed up run time, adjust if needed
max_k = 100 
param_grid = {'n_neighbors': list(range(1, max_k+1))}

gridsearch = GridSearchCV(
    KNeighborsClassifier(metric='euclidean'),
    param_grid,
    scoring='roc_auc',
    cv=kfolds,
    n_jobs=-1
)
gridsearch.fit(X_train, y_train)

clf_best = gridsearch.best_estimator_
best_k = clf_best.n_neighbors

print(f"\nOptimal number of neighbors (k): {best_k}")

# --- 5. Final Model Evaluation ---
y_test_pred_proba = clf_best.predict_proba(X_test)[:, 1]
auc_final = roc_auc_score(y_test, y_test_pred_proba)
print(f"Final KNN model (k={best_k}) Test AUC: {auc_final:.3f}")

Median Sale Price (Training Set): $163,500.00
Training data shape: (2344, 332)
Test data shape: (586, 332)

Initial KNN model (k=5) AUC: 0.969

Optimal number of neighbors (k): 14
Final KNN model (k=14) Test AUC: 0.984
